In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9


In [5]:
# Collect 50% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_50, _, y_50 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, stratify=y_label_0_1, random_state=42)

In [6]:
X_50.shape

(835636, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt50 = DecisionTreeClassifier()
# dt50.fit(X_50, y_50)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 4 5] [     7      5      1 238740]
(7, 42) (7,)
Save 1 to baseline_Def2.npy
(5, 42) (5,)
Save 2 to baseline_Def3.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(238740, 42) (238740,)
Save 5 to baseline_Def6.npy
Execution Time: 1.598612 seconds
(716259,)
[0 1 2 4 5 8 9] [660392     34  21483   1908  31576    822     44]
(660392, 42) (660392,)
Save 0 to BIM_Def1.npy
(34, 42) (34,)
Save 1 to BIM_Def2.npy
(21483, 42) (21483,)
Save 2 to BIM_Def3.npy
(1908, 42) (1908,)
Save 4 to BIM_Def5.npy
(31576, 42) (31576,)
Save 5 to BIM_Def6.npy
(822, 42) (822,)
Save 8 to BIM_Def9.npy
(44, 42) (44,)
Save 9 to BIM_Def10.npy
Execution Time: 3.067570 seconds
(716259,)
[0 1 2 4 5 6 7 8 9] [238224     46  36004  12177 414866      3   5573   1954   7412]
(238224, 42) (238224,)
Save 0 to FGSM_Def1.npy
(46, 42) (46,)
Save 1 to FGSM_Def2.npy
(36004, 42) (36004,)
Save 2 to FGSM_Def3.npy
(12177, 42) (12177,)
Save 4 to FGSM_Def5.npy
(414866, 42) (414866,)
Save 5 to FGSM_Def6.npy
(3, 42) (3,)
Save 6 to

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 4 5] [     5      1      2 238745]
(5, 42) (5,)
Save 1 to baseline_Def2.npy
(1, 42) (1,)
Save 2 to baseline_Def3.npy
(2, 42) (2,)
Save 4 to baseline_Def5.npy
(238745, 42) (238745,)
Save 5 to baseline_Def6.npy
Execution Time: 3.307143 seconds
(716259,)
[0 1 2 3 4 5 9] [203034 217690  31544      4   1366 262189    432]
(203034, 42) (203034,)
Save 0 to BIM_Def1.npy
(217690, 42) (217690,)
Save 1 to BIM_Def2.npy
(31544, 42) (31544,)
Save 2 to BIM_Def3.npy
(4, 42) (4,)
Save 3 to BIM_Def4.npy
(1366, 42) (1366,)
Save 4 to BIM_Def5.npy
(262189, 42) (262189,)
Save 5 to BIM_Def6.npy
(432, 42) (432,)
Save 9 to BIM_Def10.npy
Execution Time: 13.023759 seconds
(716259,)
[0 1 2 3 4 5 9] [222120 216228   9084     73   4987 258612   5155]
(222120, 42) (222120,)
Save 0 to FGSM_Def1.npy
(216228, 42) (216228,)
Save 1 to FGSM_Def2.npy
(9084, 42) (9084,)
Save 2 to FGSM_Def3.npy
(73, 42) (73,)
Save 3 to FGSM_Def4.npy
(4987, 42) (4987,)
Save 4 to FGSM_Def5.npy
(258612, 42) (258612,)
Save 5 to FG

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt50.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude2Attack/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude2Attack/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude2Attack/WUSTL_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_Exclude2Attack.csv")

In [13]:
all_attack_list

0       1       2       3   4      5       6   7     8      9      10  \
0   XGB50       0       7       5   0      1  238740   0     0      0      0   
1   XGB50  660392      34   21483   0   1908   31576   0     0    822     44   
2   XGB50  238224      46   36004   0  12177  414866   3  5573   1954   7412   
3   XGB50  660392      34   21483   0   1908   31576   0     0    822     44   
4   XGB50     138       0    5793   2   9254  688145  21  3252    173   9481   
5   XGB50  327914       0   58873  15   1323  314117   0  3921   8897   1199   
6   XGB50       0      25      11   0      4  716219   0     0      0      0   
7   XGB50  304295   13274   16286   3   2112  346609   0     0  30301   3379   
8   XGB50  231071       0  470968   2    634   13390   0     0    192      2   
9   XGB50  442184       0   67636   3   2085  199676   0     7    760   3908   
10   RF50       0       5       1   0      2  238745   0     0      0      0   
11   RF50  203034  217690   31544   4   1366  262189   0     0      0    432   
12   RF50  222120  216228    9084  73   4987  258612   0     0      0   5155   
13   RF50  203034  217690   31544   4   1366  262189   0     0      0    432   
14   RF50      92       0    6061  34   8526  687982  21  3273    193  10077   
15   RF50  203729     223   32907   8   1592  477787   0     0      0     13   
16   RF50       0      18       3   0      6  716232   0     0      0      0   
17   RF50  279980       0       0   0      0  436278   0     0      0      1   
18   RF50   42822  217253  430477   0      2   25705   0     0      0      0   
19   RF50  184757  193099   17692  16    892  317221   0     0      0   2582   
20   DT50       0       0       0   0      0       0   0     0      0      0   
21   DT50       0       0       0   0      0       0   0     0      0      0   
22   DT50       0       0       0   0      0       0   0     0      0      0   
23   DT50       0       0       0   0      0       0   0     0      0      0   
24   DT50       0       0       0   0      0       0   0     0      0      0   
25   DT50       0       0       0   0      0       0   0     0      0      0   
26   DT50       0       0       0   0      0       0   0     0      0      0   
27   DT50       0       0       0   0      0       0   0     0      0      0   
28   DT50       0       0       0   0      0       0   0     0      0      0   
29   DT50       0       0       0   0      0       0   0     0      0      0   

    11      12  
0    0  238753  
1    0  716259  
2    0  716259  
3    0  716259  
4    0  716259  
5    0  716259  
6    0  716259  
7    0  716259  
8    0  716259  
9    0  716259  
10   0  238753  
11   0  716259  
12   0  716259  
13   0  716259  
14   0  716259  
15   0  716259  
16   0  716259  
17   0  716259  
18   0  716259  
19   0  716259  
20   0       0  
21   0       0  
22   0       0  
23   0       0  
24   0       0  
25   0       0  
26   0       0  
27   0       0  
28   0       0  
29   0       0